In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.preprocessing import LabelEncoder
from itertools import product


import time

In [15]:
test = pd.read_csv('../data/test.csv', index_col=0)
submit = pd.read_csv('../data/sample_submission.csv')
data = pd.read_pickle('data.pkl')


In [16]:
data.columns

Index(['month_idx', 'shop_id', 'item_id', 'item_cnt_month', 'cat_id',
       'type_code', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2',
       'item_cnt_month_lag_3', 'item_cnt_month_lag_6', 'item_cnt_month_lag_12',
       'date_avg_item_cnt_lag_1', 'date_shop_avg_item_cnt_lag_1',
       'date_shop_avg_item_cnt_lag_2', 'date_shop_avg_item_cnt_lag_4',
       'date_shop_avg_item_cnt_lag_6', 'date_shop_avg_item_cnt_lag_12',
       'date_cat_avg_item_cnt_lag_1', 'date_shop_cat_avg_item_cnt_lag_1',
       'delta_price_lag', 'delta_revenue_lag_1', 'month', 'days',
       'item_last_sale', 'item_shop_first_sale', 'item_first_sale'],
      dtype='object')

In [17]:
data = data[[
'month_idx', 'shop_id', 'item_id', 'item_cnt_month', 'cat_id',
       'type_code', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2',
       'item_cnt_month_lag_3', 'item_cnt_month_lag_6', 'item_cnt_month_lag_12',
       'date_avg_item_cnt_lag_1', 'date_shop_avg_item_cnt_lag_1',
       'date_shop_avg_item_cnt_lag_2', 'date_shop_avg_item_cnt_lag_4',
       'date_shop_avg_item_cnt_lag_6', 'date_shop_avg_item_cnt_lag_12',
       'date_cat_avg_item_cnt_lag_1', 'date_shop_cat_avg_item_cnt_lag_1',
       'delta_price_lag', 'delta_revenue_lag_1', 'month', 'days',
       'item_last_sale', 'item_shop_first_sale', 'item_first_sale'
]]


In [18]:
data.month_idx.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23], dtype=int8)

In [19]:
X_train = data[data.month_idx < 21].drop(['item_cnt_month'], axis=1)
y_train = data[data.month_idx < 21]['item_cnt_month']
X_valid = data[data.month_idx == 21].drop(['item_cnt_month'], axis=1)
y_valid  = data[data.month_idx == 21]['item_cnt_month']
X_test = data[data.month_idx ==23].drop(['item_cnt_month'], axis=1)


In [20]:
del data

In [21]:
import gc


In [22]:
gc.collect()

74

In [ ]:

ts = time.time()

model = XGBRegressor(
    max_depth=8,
    n_estimators = 1000,
    min_child_weight = 300,
    colsample_bytree=0.8,
    subsample=0.8,
    eta=0.3,
    seed=42
)

model.fit(X_train, y_train,eval_metric="rmse", eval_set=[(X_train, y_train),(X_valid, y_valid)],
    verbose=True,
    early_stopping_rounds=10)

time.time() - ts


[0]	validation_0-rmse:1.75090	validation_1-rmse:1.87180
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:1.61088	validation_1-rmse:1.74599
[2]	validation_0-rmse:1.53161	validation_1-rmse:1.67891
[3]	validation_0-rmse:1.48957	validation_1-rmse:1.63874
[4]	validation_0-rmse:1.46310	validation_1-rmse:1.61912
[5]	validation_0-rmse:1.44599	validation_1-rmse:1.60741
[6]	validation_0-rmse:1.43500	validation_1-rmse:1.59611
[7]	validation_0-rmse:1.42561	validation_1-rmse:1.58980
[8]	validation_0-rmse:1.41962	validation_1-rmse:1.58730
[9]	validation_0-rmse:1.41462	validation_1-rmse:1.58438
[10]	validation_0-rmse:1.41195	validation_1-rmse:1.58338
[11]	validation_0-rmse:1.40681	validation_1-rmse:1.58283
[12]	validation_0-rmse:1.40401	validation_1-rmse:1.58239
[13]	validation_0-rmse:1.40274	validation_1-rmse:1.58242
[14]	validation_0-rmse:1.40006	validation_1-rmse:1.58

In [48]:
y_pred = model.predict(X_valid).clip(0,20)
y_test = model.predict(X_test).clip(0,20)

submission = pd.DataFrame({
    "ID": test.index,
    "item_cnt_month": y_test
})

submission.to_csv('xgb_submission.csv', index=False )


In [25]:
submit

,0,100
0,1,100
1,2,100
2,3,100
3,4,100
4,5,100
...,...,...
3054,3055,100
3055,3056,100
3056,3057,100
3057,3058,100


In [13]:
submission

,ID,item_cnt_month
0,0,0.360681
1,1,0.000000
2,2,0.383577
3,3,1.242619
4,4,2.760417
...,...,...
3055,3055,0.765410
3056,3056,0.000894
3057,3057,0.000894
3058,3058,0.000894
